In [2]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from all_staff import *

In [3]:
with open(obj_img_dir, 'rb') as f:
    objpoints, imgpoints = pickle.load(f)

In [37]:
class Line(object):
    def __init__(self, objpoints, imgpoints):
        self.do_first = True
        
        self.left_fit = None
        self.right_fit = None
        
        self.objpoints = objpoints
        self.imgpoints = imgpoints
        
        
        self.left_fitx_history = []
        self.right_fitx_history = []
        
        self.curvature_history = []
        
        self.count = 0
        self.ploty = None
        
        self.binary_warp = None
    
    def pipline(self, img):
        # update process
        self.count += 1
        
        undist = undistort_img(img, self.objpoints, self.imgpoints)
        # binary_img = binary_color(undist)
        binary_img,_  = binarize(undist)
        mask_img = mask_grad(binary_img)
        binary_warped, Minv = warp_M(mask_img)
        
        if self.do_first: # Sould use slide windows in First Frame 
            ploty, left_fitx, right_fitx, left_fit, right_fit = slide_windows(binary_warped, nwindows=9, margin=50, minpix=5)
            
            # modify the configuration 
            self.left_fit = left_fit
            self.right_fit = right_fit
            self.left_fitx_history.append(left_fitx)
            self.right_fitx_history.append(right_fitx)
            self.ploty = ploty
            
        else: # reuse left_fit and so on to acc the process
            ploty, left_fitx, right_fitx, left_fit, right_fit = acc_frame_to_frame(binary_warped, self.left_fit, self.right_fit, margin = 50)
            
            # modify the configuration
            self.left_fit = left_fit
            self.right_fit = right_fit
            self.left_fitx_history.append(left_fitx)
            self.right_fitx_history.append(right_fitx)
        
        # cal the curvature and put them into the image
        curvature = cal_curvature(ploty, left_fitx, right_fitx)
        
        # low 0.5 is a big curve
        if min(curvature)/1000 < 0.5 and not self.left_fitx_history == []:
            self.left_fitx_history.pop()
            self.right_fitx_history.pop()
            self.curvature_history.pop()
            left_fitx = self.left_fitx_history[-1]
            right_fitx = self.right_fitx_history[-1]
            curvature = self.curvature_history[-1]
        
        center_offset = cal_center_offset(left_fitx, right_fitx)
        c_s = "Curvature: left:{:.2f}km  right:{:.2f}km".format(curvature[0]/1000, curvature[1]/1000)
        c_o = "Center offset: {:.2f}m".format(center_offset)
        
        result = img_region(binary_warped, left_fitx, right_fitx, ploty, Minv, undist)
        # result_warp = img_region_no_T(binary_warped, left_fitx, right_fitx, ploty)
        
        # concat many image
#         result = cv2.resize(result, (640, 320))
#         binary_img = dstack_img(cv2.resize(binary_img, (640, 320)), b=True)
#         mask_img = dstack_img(cv2.resize(mask_img, (640, 320)), r=True)
#         result_warp = cv2.resize(result_warp, (640, 320))

#         temp1 = np.concatenate([result, result_warp], axis=0)
#         temp2 = np.concatenate([binary_img, mask_img], axis=0)
#         temp = np.concatenate([temp1, temp2], axis=1)
        
        cv2.putText(result, c_s, (22,22), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0),2)
        cv2.putText(result, c_o, (22,60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0), 2)
        
        self.curvature_history.append(curvature)
            
        return result
        

In [38]:
l = Line(objpoints, imgpoints)

In [41]:
white_output = 'video/project_video_res_harder.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("video/challenge_video.mp4").subclip(0,5)
white_clip = clip1.fl_image(l.pipline) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video video/project_video_res_harder.mp4
[MoviePy] Writing video video/project_video_res_harder.mp4


100%|██████████| 150/150 [02:09<00:00,  1.14it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: video/project_video_res_harder.mp4 

CPU times: user 2min 15s, sys: 5.5 s, total: 2min 20s
Wall time: 2min 11s


In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## 接下来要做

- 换threshold算法来避免光照-黑暗的场景替换
- smoothing，计算变化率，曲率，leftcenter
- 前几个frame的变化率比较